In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
from scipy.stats import mode

temperature = pd.read_csv("기온.csv", encoding="cp949")
rainfall = pd.read_csv("강수량.csv", encoding="cp949")
wind = pd.read_csv("바람.csv", encoding="cp949")
humidity = pd.read_csv("습도.csv", encoding="cp949")
sunshine = pd.read_csv("일조일사.csv", encoding="cp949")

temperature = temperature.drop([temperature.columns[0], temperature.columns[1]], axis=1)
rainfall = rainfall.drop([rainfall.columns[0], rainfall.columns[1]], axis=1)
wind = wind.drop([wind.columns[0], wind.columns[1]], axis=1)
humidity = humidity.drop([humidity.columns[0], humidity.columns[1]], axis=1)
sunshine = sunshine.drop([sunshine.columns[0], sunshine.columns[1]], axis=1)

temperature.rename(columns = {"\t최고기온시각": "최고기온시각"}, inplace=True)
wind.rename(columns = {"\t최대풍속시각":"최대풍속시각", "\t최대순간풍속시각": "최대순간풍속시각"}, inplace=True)

merge_data = pd.merge(temperature, rainfall, on='일시')
merge_data = pd.merge(merge_data, wind, on='일시')
merge_data = pd.merge(merge_data, humidity, on='일시')
merge_data = pd.merge(merge_data, sunshine, on='일시')
merge_data = merge_data[:2158]

merge_data.to_csv("observed.csv", mode='w', encoding='cp949')
observed = pd.read_csv("observed.csv", encoding='cp949')
observed = observed.drop([observed.columns[0]], axis=1) # 저장하면서 index열이 생김. 이를 지우기 위함.

weather = pd.read_csv("기상(타겟).csv", encoding="cp949", low_memory=False) # low_memory = 경고 무시용

weather = weather[weather['지역']=='충청북도']
weather = weather.drop(weather.columns[0], axis=1)

weather = weather.sort_values(by=['예보시각'])
weather = weather.reset_index(drop=True)

forecast_score = {'맑음':0,
                  '구름조금':1, '구름많음':1.1,
                  '흐림':2,
                  '구름많고 소나기':3, '구름많고 비':3.1, '구름많고 비/눈':3.2, '구름많고 눈/비':3.2, '구름많고 눈':3.3,
                  '흐리고 비':4, '흐리고 눈/비':4.1, '흐리고 비/눈':4.1, '흐리고 눈':4.2}
re_score = {v:k for k,v in forecast_score.items()}
from scipy.stats import mode

def select_forecast(date):
    temp = 0
    for forecast in date:
        if forecast_score[forecast] > temp:
            temp = forecast_score[forecast]
    return re_score[temp]
weather_copy = weather.copy()
for i, w in enumerate(weather_copy['예보시각']):
    weather_copy['예보시각'][i] = w[:10]
    
weather_copy = weather_copy.drop_duplicates(['예보시각'])
weather_copy = weather_copy.reset_index(drop=True)
weather_copy['예보'] = ''

w=0
start = 0
for i in range(len(weather)):
    if weather_copy["예보시각"][w] not in weather["예보시각"][i]:
        weather_copy["예보"][w] = select_forecast(weather["예보"][start:i])
        w += 1
        start = i
        
weather = weather_copy

weather = weather.drop([weather.columns[0]], axis=1)

weather.to_csv("forecast.csv", mode='w', encoding='cp949')
forecast = pd.read_csv("forecast.csv", encoding='cp949')
forecast = forecast.drop([forecast.columns[0]], axis=1) # 저장하면서 index열이 생김. 이를 지우기 위함.

forecast.rename(columns = {"예보시각": "일시"}, inplace=True)
df = pd.merge(observed, forecast, on='일시')
df.to_csv("df.csv", encoding='cp949')
df = pd.read_csv("df.csv", encoding='cp949')
df = df.drop([df.columns[0]], axis=1) # 저장하면서 index열이 생김. 이를 지우기 위함.

df


,일시,평균기온(℃),최고기온(℃),최고기온시각,최저기온(℃),최저기온시각,일교차,강수량(mm),1시간최다강수량(mm),1시간최다강수량시각,...,최대풍속시각,최대순간풍속(m/s),최대순간풍속풍향(deg),최대순간풍속시각,평균습도(%rh),최저습도(%rh),일조합,일조율,일사합,예보
0,2016-01-01,-1.0,4.1,14:39,-5.9,3:54,10.0,NaN,NaN,NaN,...,15:47,3.2,20.0,15:38,79.4,53.0,3.1,32.0,NaN,구름조금
1,2016-01-02,3.5,9.9,13:07,0.4,5:08,9.5,NaN,NaN,NaN,...,13:43,3.7,340.0,14:36,74.9,52.0,2.6,26.8,NaN,구름많고 눈/비
2,2016-01-03,3.6,11.4,15:07,-1.6,5:43,13.0,NaN,NaN,NaN,...,12:32,3.0,230.0,12:51,81.0,53.0,5.0,51.5,NaN,구름많음
3,2016-01-04,3.9,8.0,13:45,-2.8,23:51,10.8,NaN,NaN,NaN,...,14:57,7.5,250.0,13:55,54.3,27.0,6.4,66.0,NaN,구름많음
4,2016-01-05,-2.0,2.7,15:05,-6.1,23:57,8.8,NaN,NaN,NaN,...,15:53,6.3,270.0,15:50,46.5,17.0,6.0,61.9,NaN,구름많음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2153,2021-11-23,0.0,1.6,14:10,-2.8,7:31,4.4,NaN,NaN,NaN,...,15:42,8.9,290.0,13:04,65.0,41.0,0.0,0.0,1.93,흐리고 눈
2154,2021-11-24,1.4,6.0,14:44,-1.3,3:26,7.3,0.0,NaN,NaN,...,1:42,6.8,290.0,1:33,71.9,50.0,0.5,5.0,2.44,흐리고 눈
2155,2021-11-25,5.2,11.5,14:08,-0.9,23:48,12.4,0.0,NaN,NaN,...,14:16,8.8,250.0,14:28,69.8,36.0,6.5,65.7,8.03,구름많고 비
2156,2021-11-26,1.8,8.8,13:44,-3.8,8:02,12.6,NaN,NaN,NaN,...,15:54,7.0,360.0,14:38,64.8,33.0,6.0,60.6,9.46,맑음
